# Untyped Lambda Calculus

In [1]:
from copy import deepcopy
from nanoid import generate


In [2]:
def genid(n=6):
    s = "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper()
    return generate(s, n)


In [3]:
genid()


'ZyIBcB'

In [4]:
class parser:
    """
    # S-Exp Parser
        Grammar:
            P -> id | E
            L -> P | P L
            E -> (L)

        LL(1):
            P  -> id | E
            L  -> PL'
            L' -> L | null
            E  -> (L)
    """

    def lexer(s):
        s = s.replace("\n", " ").replace("(", " ( ").replace(")", " ) ")
        l = s.split()
        ret = []
        for c in l:
            if c == "(":
                ret.append(("(", None))
            elif c == ")":
                ret.append((")", None))
            else:
                ret.append(("id", c))
        return ret

    def __init__(self, s):
        self.tokens = list(reversed(parser.lexer(s)))

    def run(self):
        return self.E()

    def next(self):
        self.tokens.pop()

    def peek(self):
        return self.tokens[-1]

    def P(self):
        token = self.peek()
        if token[0] == "id":
            # print("P id={}".format(token[1]))
            self.next()
            return token[1]
        else:
            ret = self.E()
            return ret

    def L(self):
        token = self.peek()
        ret = [self.P()]
        ret += self.Lp()
        return ret

    def Lp(self):
        token = self.peek()
        if token[0] == "id" or token[0] == "(":
            ret = self.L()
            return ret
        else:
            return []

    def E(self):
        token = self.peek()
        assert token[0] == "("
        self.next()
        ret = self.L()
        token = self.peek()
        assert token[0] == ")"
        self.next()
        return ret


# NbE

In [5]:
# Normal Forms of Untyped λ-Calculus
#  ‹expr› ::= ‹id›
#          |  ( λ ( ‹id› ) ‹expr› )
#          |  ( ‹expr› ‹expr› )
# LL1:
# ‹norm›::= ‹neu›|( λ ( ‹id› ) ‹norm› )
# ‹neu›::=‹id›|( ‹neu› ‹norm› )


In [38]:
class nbe_runner:
    def __init__(self, slist):
        self.prog = slist
        self.indent = 0

    def clos(self, env, var, body):
        return ["CLOS", env, var, body]

    def assv(self, name, env):
        try:
            return env[name]
        except:
            return False

    def extend(self, env, name, val):
        ret = deepcopy(env)
        ret[name] = val
        return ret

    def is_symbol(self, s):
        if type(s) == str:
            return True
        return False

    def freshen(self, used, x):
        print("FRESHEN", str(used), x)
        if x in used:
            return self.freshen(used, x + "*")
        else:
            return x

    def lazy(self, env, exp):
        return ["LAZY", env, exp]

    def n_var(self, x):
        return ["N-var", x, genid()]

    def n_ap(self, rator, rand):
        return ["N-ap", rator, rand]

    def val(self, env, exp):
        print("  " * self.indent, end="")
        print("EVAL", exp, "in", env)
        self.indent += 1

        ret = None
        match exp:
            case ["λ", [var], body]:
                ret = self.clos(env, var, body)
            # case ["LAZY", env, exp]:
            #     return self.val(env, exp)
            case [rator, rand]:
                ret = self.do_ap(self.val(env, rator), self.val(env, rand))
            case x:
                xv = self.assv(x, env)
                if xv != False:
                    ret = xv
                else:
                    # raise RuntimeError("Unknown variable {}".format(exp))
                    return self.n_var(x)
        self.indent -= 1
        return ret

    def do_ap(self, fun, arg):
        print("  " * self.indent, end="")
        print("APPLY", fun, "to", arg)
        self.indent += 1

        ret = None
        match fun:
            case ["CLOS", env, var, body]:
                nenv = self.extend(env, var, arg)
                # ret = self.val(nenv, body)
                ret = self.lazy(nenv, body)
            case ["LAZY", env, exp]:
                f = self.val(env, exp)
                ret = self.do_ap(f, arg)
            case _:
                ret = self.n_ap(fun, arg)
        self.indent -= 1
        return ret

    def read_back(self, used_names, v):
        print("  " * self.indent, end="")
        print("READBACK", used_names, v)
        self.indent += 1
        ret = None
        match v:
            case ["CLOS", env, x, body]:
                y = self.freshen(used_names, x)
                ny = self.n_var(y)
                ret = [
                    "λ",
                    [y],
                    self.read_back(
                        used_names + [y], self.val(self.extend(env, x, ny), body)
                    ),
                ]
            case ["LAZY", env, exp]:
                # raise RuntimeError
                x = self.val(env, exp)
                ret = self.read_back(used_names, x)
            case ["N-var", var, gid]:
                # ret = var+'_'+gid
                ret = var
            case ["N-ap", rator, rand]:
                ret = [
                    self.read_back(used_names, rator),
                    self.read_back(used_names, rand),
                ]
            case _:
                raise RuntimeError("Read Back Failed")
        self.indent -= 1
        return ret

    def norm(self, env, exp):
        print("  " * self.indent, end="")
        print("NORM", str(exp))

        self.indent += 1
        ret = self.read_back([], self.val(env, exp))
        self.indent -= 1
        return ret

    def _run(self, env, exprs):
        self.loc += 1
        if len(exprs) > 0:
            exp = exprs[0]
            rest = exprs[1:]
            if exp[0] == "define":
                x = exp[1]
                e = exp[2]
                v = self.val(env, e)
                self._run(self.extend(env, x, v), rest)
            else:
                print("#", self.loc, ">", self.norm(env, exp))
                self._run(env, rest)
        return

    def run(self):
        self.loc = 0
        self._run({}, self.prog)


class nbe_interpretor:
    def __init__(self, prog):
        p = parser(prog)
        slist = p.run()
        self.exec = nbe_runner(slist)

    def run(self):
        self.exec.run()


In [39]:
prog04 = """
(
    (
        (λ (x) y) 
        (
            (λ (x) (x x))
            (λ (x) (x x))
        )
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


NORM [['λ', ['x'], 'y'], [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]]]
  EVAL [['λ', ['x'], 'y'], [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]]] in {}
    EVAL ['λ', ['x'], 'y'] in {}
    EVAL [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]] in {}
      EVAL ['λ', ['x'], ['x', 'x']] in {}
      EVAL ['λ', ['x'], ['x', 'x']] in {}
      APPLY ['CLOS', {}, 'x', ['x', 'x']] to ['CLOS', {}, 'x', ['x', 'x']]
    APPLY ['CLOS', {}, 'x', 'y'] to ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]
  READBACK [] ['LAZY', {'x': ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]}, 'y']
    EVAL y in {'x': ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]}
      READBACK [] ['N-var', 'y', 'rRZXCq']
# 1 > y


In [40]:
prog04 = """
(
    (
        (λ (x) ((λ (x) x) x)) x
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


NORM [['λ', ['x'], [['λ', ['x'], 'x'], 'x']], 'x']
  EVAL [['λ', ['x'], [['λ', ['x'], 'x'], 'x']], 'x'] in {}
    EVAL ['λ', ['x'], [['λ', ['x'], 'x'], 'x']] in {}
    EVAL x in {}
      APPLY ['CLOS', {}, 'x', [['λ', ['x'], 'x'], 'x']] to ['N-var', 'x', 'VFbncG']
    READBACK [] ['LAZY', {'x': ['N-var', 'x', 'VFbncG']}, [['λ', ['x'], 'x'], 'x']]
      EVAL [['λ', ['x'], 'x'], 'x'] in {'x': ['N-var', 'x', 'VFbncG']}
        EVAL ['λ', ['x'], 'x'] in {'x': ['N-var', 'x', 'VFbncG']}
        EVAL x in {'x': ['N-var', 'x', 'VFbncG']}
        APPLY ['CLOS', {'x': ['N-var', 'x', 'VFbncG']}, 'x', 'x'] to ['N-var', 'x', 'VFbncG']
      READBACK [] ['LAZY', {'x': ['N-var', 'x', 'VFbncG']}, 'x']
        EVAL x in {'x': ['N-var', 'x', 'VFbncG']}
        READBACK [] ['N-var', 'x', 'VFbncG']
# 1 > x


In [41]:
testrb = nbe_runner([])
testrb.norm({}, [["λ", ["x"], ["λ", ["y"], ["x", "y"]]], ["λ", ["x"], "x"]])


NORM [['λ', ['x'], ['λ', ['y'], ['x', 'y']]], ['λ', ['x'], 'x']]
  EVAL [['λ', ['x'], ['λ', ['y'], ['x', 'y']]], ['λ', ['x'], 'x']] in {}
    EVAL ['λ', ['x'], ['λ', ['y'], ['x', 'y']]] in {}
    EVAL ['λ', ['x'], 'x'] in {}
    APPLY ['CLOS', {}, 'x', ['λ', ['y'], ['x', 'y']]] to ['CLOS', {}, 'x', 'x']
  READBACK [] ['LAZY', {'x': ['CLOS', {}, 'x', 'x']}, ['λ', ['y'], ['x', 'y']]]
    EVAL ['λ', ['y'], ['x', 'y']] in {'x': ['CLOS', {}, 'x', 'x']}
    READBACK [] ['CLOS', {'x': ['CLOS', {}, 'x', 'x']}, 'y', ['x', 'y']]
FRESHEN [] y
      EVAL ['x', 'y'] in {'x': ['CLOS', {}, 'x', 'x'], 'y': ['N-var', 'y', 'qhaRas']}
        EVAL x in {'x': ['CLOS', {}, 'x', 'x'], 'y': ['N-var', 'y', 'qhaRas']}
        EVAL y in {'x': ['CLOS', {}, 'x', 'x'], 'y': ['N-var', 'y', 'qhaRas']}
        APPLY ['CLOS', {}, 'x', 'x'] to ['N-var', 'y', 'qhaRas']
      READBACK ['y'] ['LAZY', {'x': ['N-var', 'y', 'qhaRas']}, 'x']
        EVAL x in {'x': ['N-var', 'y', 'qhaRas']}
        READBACK ['y'] ['N-var', 'y

['λ', ['y'], 'y']

In [42]:
prog02 = """
((define z
       (λ (f)
         (λ (x)
           x)))
     (define s
       (λ (n)
         (λ (f)
           (λ (x)
             (f ((n f) x))))))
     (s (s z))))
"""
test = nbe_interpretor(prog02)
test.run()


EVAL ['λ', ['f'], ['λ', ['x'], 'x']] in {}
EVAL ['λ', ['n'], ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]] in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}
NORM ['s', ['s', 'z']]
  EVAL ['s', ['s', 'z']] in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 's': ['CLOS', {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n', ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]]}
    EVAL s in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 's': ['CLOS', {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n', ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]]}
    EVAL ['s', 'z'] in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 's': ['CLOS', {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n', ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]]}
      EVAL s in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 's': ['CLOS', {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n', ['λ', ['f'], ['λ', ['x'], ['f', [['n', 'f'], 'x']]]]]}
      EVAL z in {'z': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 's': ['CLOS', {'z': ['

# NbE Play

In [43]:
def with_numerals(exp):
    s = (
        """((define ZERO
      (λ (f)
        (λ (x)
          x)))
    (define ADD1
      (λ (n-1)
        (λ (f)
          (λ (x)
            (f ((n-1 f) x))))))
    """
        + exp
        + ")"
    )
    ret = parser(s).run()
    # ret = s
    return ret


def to_church(n):
    if n <= 0:
        return "ZERO"
    else:
        return "(ADD1 {})".format(to_church(n - 1))


def church_add(i, j):
    return """
    (((λ (j)
     (λ (k)
       (λ (f)
         (λ (x)
           ((j f) ((k f) x)))))) {}) {})
    """.format(
        i, j
    )


In [44]:
prog02 = with_numerals(church_add(to_church(2), to_church(3)))
for i, s in enumerate(prog02):
    print(i + 1, s)
print()
testrb = nbe_runner(prog02)
testrb.run()


1 ['define', 'ZERO', ['λ', ['f'], ['λ', ['x'], 'x']]]
2 ['define', 'ADD1', ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]]
3 [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', ['ADD1', 'ZERO']]], ['ADD1', ['ADD1', ['ADD1', 'ZERO']]]]

EVAL ['λ', ['f'], ['λ', ['x'], 'x']] in {}
EVAL ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]] in {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}
NORM [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', ['ADD1', 'ZERO']]], ['ADD1', ['ADD1', ['ADD1', 'ZERO']]]]
  EVAL [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', ['ADD1', 'ZERO']]], ['ADD1', ['ADD1', ['ADD1', 'ZERO']]]] in {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 'ADD1': ['CLOS', {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n-1', ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]}
    EVAL [['λ', 

In [45]:
prog03 = """
(
    (((λ (f) (λ (x) (f x))) x) z)
)
"""
test = nbe_interpretor(prog03)
test.run()


NORM [[['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'], 'z']
  EVAL [[['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'], 'z'] in {}
    EVAL [['λ', ['f'], ['λ', ['x'], ['f', 'x']]], 'x'] in {}
      EVAL ['λ', ['f'], ['λ', ['x'], ['f', 'x']]] in {}
      EVAL x in {}
        APPLY ['CLOS', {}, 'f', ['λ', ['x'], ['f', 'x']]] to ['N-var', 'x', 'IIekII']
      EVAL z in {}
        APPLY ['LAZY', {'f': ['N-var', 'x', 'IIekII']}, ['λ', ['x'], ['f', 'x']]] to ['N-var', 'z', 'FKCyDW']
          EVAL ['λ', ['x'], ['f', 'x']] in {'f': ['N-var', 'x', 'IIekII']}
          APPLY ['CLOS', {'f': ['N-var', 'x', 'IIekII']}, 'x', ['f', 'x']] to ['N-var', 'z', 'FKCyDW']
      READBACK [] ['LAZY', {'f': ['N-var', 'x', 'IIekII'], 'x': ['N-var', 'z', 'FKCyDW']}, ['f', 'x']]
        EVAL ['f', 'x'] in {'f': ['N-var', 'x', 'IIekII'], 'x': ['N-var', 'z', 'FKCyDW']}
          EVAL f in {'f': ['N-var', 'x', 'IIekII'], 'x': ['N-var', 'z', 'FKCyDW']}
          EVAL x in {'f': ['N-var', 'x', 'IIekII'], 'x': ['N-var', 'z',

In [33]:
prog04 = """
(
    (
        (λ (x) y) 
        (
            (λ (x) (x x))
            (λ (x) (x x))
        )
    )
)
"""
test = nbe_interpretor(prog04)
test.run()


NORM [['λ', ['x'], 'y'], [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]]]
  EVAL [['λ', ['x'], 'y'], [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]]] in {}
    EVAL ['λ', ['x'], 'y'] in {}
    EVAL [['λ', ['x'], ['x', 'x']], ['λ', ['x'], ['x', 'x']]] in {}
      EVAL ['λ', ['x'], ['x', 'x']] in {}
      EVAL ['λ', ['x'], ['x', 'x']] in {}
      APPLY ['CLOS', {}, 'x', ['x', 'x']] to ['CLOS', {}, 'x', ['x', 'x']]
    APPLY ['CLOS', {}, 'x', 'y'] to ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]
  READBACK [] ['LAZY', {'x': ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]}, 'y']
    EVAL y in {'x': ['LAZY', {'x': ['CLOS', {}, 'x', ['x', 'x']]}, ['x', 'x']]}
      READBACK [] ['N-var', 'y', 'EUUfzN']
# 1 > y_EUUfzN


In [34]:
prog02 = with_numerals(church_add(to_church(1), to_church(1)))
for i, s in enumerate(prog02):
    print(i + 1, s)
print()
testrb = nbe_runner(prog02)
testrb.run()

1 ['define', 'ZERO', ['λ', ['f'], ['λ', ['x'], 'x']]]
2 ['define', 'ADD1', ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]]
3 [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', 'ZERO']], ['ADD1', 'ZERO']]

EVAL ['λ', ['f'], ['λ', ['x'], 'x']] in {}
EVAL ['λ', ['n-1'], ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]] in {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}
NORM [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', 'ZERO']], ['ADD1', 'ZERO']]
  EVAL [[['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1', 'ZERO']], ['ADD1', 'ZERO']] in {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']], 'ADD1': ['CLOS', {'ZERO': ['CLOS', {}, 'f', ['λ', ['x'], 'x']]}, 'n-1', ['λ', ['f'], ['λ', ['x'], ['f', [['n-1', 'f'], 'x']]]]]}
    EVAL [['λ', ['j'], ['λ', ['k'], ['λ', ['f'], ['λ', ['x'], [['j', 'f'], [['k', 'f'], 'x']]]]]], ['ADD1'